## Занятие 4. Разработка ml проектов: хорошие и плохие практики
На занятии 3 мы написали пайплайн обучения семантического сегментатора подводных фото в одном jupyter ноутбуке.

На этом занятии мы оформим этот пайплайн в ml проект так, чтобы его можно было поддерживать и развивать.


In [1]:
from pathlib import Path
ROOT_PATH = Path().absolute()
assert ROOT_PATH.name == 'c04', ROOT_PATH.name
ROOT_PATH

### Плохая практика 1: использование jupyter ноутбуков в качестве основного средства разработки
- https://www.kdnuggets.com/2019/11/notebook-anti-pattern.html
- https://analyticsindiamag.com/an-argument-against-using-jupyter-notebook-for-machine-learning/
- https://medium.com/skyline-ai/jupyter-notebook-is-the-cancer-of-ml-engineering-70b98685ee71

Главные проблемы:
- нелинейность исполнения кода (сложно читать и сложно воспроизводить)
- невозможность тестирования
- нечитыемые diff-ы в git (сложно разобраться в истории и работать совместно)

Jupyter ноутбуки хорошо подходят чтобы:
- визуализировать данные
- набросать прототип
- поделиться однократным результатом


#### **Решение**: 
Использовать стандартные инструменты разработки ЯП, например - Python пакеты.

In [1]:
# Python ищет пакеты в PYTHONPATH, если пакет не найден - то получаем ModuleNotFound
from suim_segmentation.data import SuimDataset

ModuleNotFoundError: No module named 'suim_segmentation'

In [2]:
cd package

D:\edu\teach\course_cvdl\classes\c04\package


In [92]:
# В PYTHONPATH всегда содержится current_dir, так что из родительской директории можно импортировать пакет
from suim_segmentation.data import SuimDataset

In [6]:
from suim_segmentation import data as suim_data
suim_data.__file__

'D:\\edu\\teach\\course_cvdl\\classes\\c04\\package\\suim_segmentation\\data.py'

Если пакет находится в текущей папке, то его можно импортировать.

Это не очень удобно - код можно будет вызывать только при определенной текущей директории.

**Правильное решение - сделать пакет устанавливаемым!**

### 1.1 Пакетируем код

На сегодня (2022 год) рекомендуется пакетировать код с помощью `pyproject.toml` файла:
- поддерживает разные системы сборки (не только setuptools)
- поддерживает C++ расширения
- стандартизирован в PEP-518, PEP-621

Есть и другие [способы](https://packaging.python.org/en/latest/glossary/?highlight=setup.py#term-setup.py), которые пока более распространены, но `pyproject.toml` является рекомендуемым

In [2]:
ls package

 Volume in drive D is Datadrive
 Volume Serial Number is 848A-4A05

 Directory of D:\edu\teach\course_cvdl\classes\c04\package

07.10.2022  18:41    <DIR>          .
07.10.2022  18:41    <DIR>          ..
07.10.2022  15:49    <DIR>          .ipynb_checkpoints
07.10.2022  18:05                 0 pyproject.toml
07.10.2022  17:27    <DIR>          suim_segmentation
07.10.2022  18:40    <DIR>          tmp
               1 File(s)              0 bytes
               5 Dir(s)  35В 990В 568В 960 bytes free


In [43]:
%%writefile {str(ROOT_PATH / 'package' / 'pyproject.toml')}

[project]
name = "suim_segmentation"
description = "ML project example package"
version = "0.1.0"

Overwriting D:\edu\teach\course_cvdl\classes\c04\package\pyproject.toml


### Задачка 1
Пользуясь гайдом https://packaging.python.org/en/latest/tutorials/packaging-projects/#creating-pyproject-toml, заполните pyproject.toml так, чтобы пакет можно было установить.

*Возможно, нужно будет обновить pip: `python -m pip install -U pip`*

Пакеты c `pyproject.toml` (или `setup.py`) можно устанавливать через git:
`TODO`

### 1.1 Проверяем код на PEP-8 с помощью `pylint`
Pylint - инструмент для проверки кода на соответствие PEP-8.

`pip install pylint`

Вывод всех нарушений PEP8 и оценки вашего кода: `python -m pylint suim_segmentation`

Вывод только ошибок: `python -m pylint suim_segmentation/ -E`

In [19]:
cd ..\package

D:\edu\teach\course_cvdl\classes\c04


In [ ]:
! python -m pylint suim_segmentation

### 1.2 Форматируем код с помощью `black`
Часть ошибок форматирования можно поправить автоматически с помощью black.
`pip install black`

Форматирование: `python -m black suim_segmentation/`

In [ ]:
! python -m black suim_segmentation/

In [ ]:
! python -m pylint suim_segmentation

### 1.3 Тюним правил PEP-8 под свой проект
Для pylint можно создать набор правил, по умолчанию - файл `.pylintrc`

In [45]:
str(ROOT_PATH / 'package' / '.pylintrc')

'D:\\edu\\teach\\course_cvdl\\classes\\c04\\package\\.pylintrc'

In [44]:
%%writefile {str(ROOT_PATH / 'package' / '.pylintrc')}

[TYPECHECK]
# ignore torch member warnings - they are broken
generated-members=numpy.*, torch.*

[FORMAT]
# allow x as a good name
good-names=i,j,k,x
# increase max length
max-line-length=128

Overwriting D:\edu\teach\course_cvdl\classes\c04\package\.pylintrc


In [ ]:
! python -m pylint suim_segmentation

### 1.4 Пишем "точку входа" в пайплайн (скрипт обучения) 

In [46]:
%%writefile {str(ROOT_PATH / 'package' / 'suim_segmentation' / 'run.py')}

import argparse
from pathlib import Path

import torch
from torch.utils import data as tdata
from tqdm import tqdm

from .data import SuimDataset, EveryNthFilterSampler
from .model import SuimModel
from .loss import DiceLoss
from .metrics import Accuracy
from .trainer import Trainer


def run_pipeline(args):
    device = None
    model = None
    opt = None

    train_val_ds = None
    test_ds = None
    
    test_iter = None
    train_iter = None
    val_iter = None
    
    loss = None
    metric = None
    
    trainer = Trainer(
        net=model,
        opt=opt,
        train_loader=train_iter,
        val_loader=val_iter,
        test_loader=test_iter,
        loss=loss,
        metric=metric,
    )
    for e in tqdm(range(args.num_epochs)):
        print(f"Epoch {e}")
        epoch_stats = trainer(1)


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--name", type=str, required=True)
    parser.add_argument("--train-data", type=str, required=True)
    parser.add_argument("--test-data", type=str, required=True)
    parser.add_argument("--lr", type=float, required=True)
    parser.add_argument("--num-epochs", type=int, required=True)
    parser.add_argument("--batch-size", type=int, default=16)
    parser.add_argument("--device", type=str, default='cpu:0')
    return parser.parse_args()


if __name__ == "__main__":
    args = parse_args()
    run_pipeline(args)
    print("Finished")

Overwriting D:\edu\teach\course_cvdl\classes\c04\package\suim_segmentation\run.py


### Задачка 2
Пользуясь ноутбуком с предыдущего занятия, допишите `suim_segmentation/run.py`, чтобы тренировка корректно запускалась.


In [26]:
! python -m suim_segmentation.run --name=baseline --lr=0.03 --num-epochs=0 --batch-size=32 --device=cuda:0 \
    --train-data=D:\edu\teach\course_cvdl\classes\c03\suim_dataset\train_val\train_val \
    --test-data=D:\edu\teach\course_cvdl\classes\c03\suim_dataset\TEST\TEST 

Testing:
Stats: Loss=0.93 Metric=0.31



0it [00:00, ?it/s]
0it [00:00, ?it/s]
C:\Users\bzimka\Anaconda3\envs\torchedu\lib\site-packages\segmentation_models_pytorch\base\modules.py:116: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)


### Плохая практика 2: "ручное" копирование данных

*Из с03/README.md:*

`скачать датасет SUIM с GoogleDrive: https://drive.google.com/drive/folders/10KMK0rNB43V2g30NcA1RYipL535DuZ-h`

- В реальных проектах (индустрия/соревнования) часто используется несколько датасетов, а не один
- Они могут быть в разных форматах, а код проекта обычно работает только с одним форматом
- Датасет может обновляться - даже у ImageNet есть [version2](https://proceedings.mlr.press/v97/recht19a/recht19a.pdf)
- Часто датасет нужен на разных машинах (например, локальной и devbox с GPU)

#### **Решение**: 
Использовать инструменты для управления данными, например - [dvc](https://dvc.org)

![dvc scheme was not loaded](dvc_scheme.png "DVC scheme")

### 2.1 Добавляем google-drive в качестве хранилища

https://dvc.org/doc/user-guide/how-to/setup-google-drive-remote

In [8]:
! dvc remote add mygoogledrive gdrive://1_8FVmJgPW-dwYr8jOe9PQupCEy53WQ4d

In [9]:
! dvc remote list

mygoogledrive	gdrive://1_8FVmJgPW-dwYr8jOe9PQupCEy53WQ4d


Добавим test-данные в индекс dvc

In [7]:
import shutil

source_dataset = ROOT_PATH.parent / 'c03' / 'suim_dataset' / 'test'
assert source_dataset.exists()

target_dataset = ROOT_PATH / 'data'/ 'test'
shutil.copytree(str(source_dataset), str(target_dataset))

'D:\\edu\\teach\\course_cvdl\\classes\\c04\\data\\test'

In [9]:
! dvc add test


To track the changes with git, run:

	git add .gitignore test.dvc

To enable auto staging, run:

	dvc config core.autostage true


Отправим данные в хранилище

In [ ]:
! dvc push test.dvc --remote=mygoogledrive

Сохраним ярлыки в git
`TODO`

### Плохая практика 3: проведение экспериментов без отслеживания результатов
Достижение лучшх результатов в любой ml-задаче требуют множество экспериментов, каждый из которых дает небольшое улучшение (или ухудшение).

Результаты экспериментов необходимо сравнивать между собой, чтобы оставлять успешные идеи и отбрасывать неудачные.

Простейший (ручной) способ отслеживания экспериментов:
- выполнили эксперимент N, запомнили метрики
- выполнили эксперимент N+1, сравнили метрики с N, запомнили
- выполнили эксперимент N+2, сравнили метрики с N+1, запомнили
- ...

Главная проблема: **после эксперимента не остаётся следов (артефактов)**

Проблемы-следствия:
- нельзя сравнить результаты эксперименты N и (N+10) 
- сложно провести многовариантный, а не бинарный эксперимент
- сложно воспроизвести идею N (если она стала снова актуальной)
- нужно держать в голове, насколько хорощо/плохо сработала идея когда-то в прошлом

**Решение:** Логировать все параметры и результаты эксперимента

### 3.1 Подключаем Weights & Biases
https://docs.wandb.ai/quickstart

1. Устанавливаем wandb: `! pip install wandb`
2. Авторизуемся на `https://wandb.ai/login` (например, через GitHub)
3. Заходим на `https://wandb.ai/settings`, копируем ключ из `API keys`
4. Выполняем `$ wandb login <YOUR API KEY>` 

### 3.2 Проверяем wnb

In [47]:
import wandb

In [76]:
wandb.init(project='hello_world', config={'lr': 0.01, 'foo': 'bar', 'something': True}, name='second')

Можно логировать численные величины. Каждое вызов log неявно увеличивает внутренний счётчик шагов.

In [79]:
wandb.log({"train": {"loss": 0.9, "metric": 0.5}, "val": {"loss": 0.4, "acc": 0.8}})

In [80]:
wandb.log({"train": {"loss": 0.8, "metric": 0.5}, "val": {"loss": 0.35, "acc": 0.8}})

In [81]:
wandb.log({"train": {"loss": 0.75, "metric": 0.52}, "val": {"loss": 0.33, "acc": 0.7}})

In [82]:
wandb.log({"train": {"loss": 0.74, "metric": 0.52}, "val": {"loss": 0.32, "acc": 0.7}})

Можно явно указать шаг, к которому относится запись

In [83]:
wandb.log({"train": {"loss": 0.74, "metric": 0.52}, "val": {"loss": 0.32, "acc": 0.7}}, step=20)

Каждый вызов log добавляет аргументы во внутреннее состояние и коммитит **предыдущие** значения.

Можно считать, что каждый вызов .log - это `git commit` старых данных + `git add` новых данных.

In [84]:
wandb.log({"train": {"loss": 0.74, "metric": 0.52}, "val": {"loss": 0.32, "acc": 0.7}}, step=50)

Можно логировать не только численные величины - например, изображения с масками.

In [85]:
from suim_segmentation.data import SuimDataset

test_data = SuimDataset(root=ROOT_PATH / 'data' / 'test' / 'TEST', masks_as_color=False)

In [86]:
x_img, y_mask = test_data[2]

In [87]:
SuimDataset.LABEL_COLORS

(('Background(waterbody)', '000'),
 ('Human divers', '001'),
 ('Aquatic plants and sea-grass', '010'),
 ('Wrecks and ruins', '011'),
 ('Robots (AUVs/ROVs/instruments)', '100'),
 ('Reefs and invertebrates', '101'),
 ('Fish and vertebrates', '110'),
 ('Sea-floor and rocks', '111'))

In [88]:
class_labels = dict(
    (num, cls_name) for num, (cls_name, binary_idx) in enumerate(SuimDataset.LABEL_COLORS)
)
mask_img = wandb.Image(x_img.permute(1, 2, 0).numpy(), masks={
  "predictions": {
    "mask_data": y_mask[0].numpy(),
    "class_labels": class_labels
  }
})

In [89]:
wandb.log({"gt_example": mask_img})

Можно добавить ключ-значение в summary

In [90]:
wandb.run.summary['my_key'] = 'my_important_value'

In [91]:
wandb.finish()

my_key,my_important_value


### 3.3 Логируем параметры и результаты эксперимента
### Задачка 3
Дописать код run.py так, чтобы логировались (как минимум):
- Средние train.loss, train.metric, val.loss, val.metric для каждой эпохт
- Средние test.loss, test.metric однократно

Сделать проект публичным, скинуть ссылку на свой wandb отчет